This notebook is based largely on the prior works of Patrick Pantel and Dekang Lin (1999). 

The primary purpose here is to classify a given sentence as having prepositional phrase attachment ambiguity. A syntactic parse is required. 

Secondary purpose will be to enumerate the candidate attachment sites for those sentences with ambiguities. 

Once this work is completed, we can use it to analyze sentences with prepositional phrases, both with and without ambiguities, and look for interesting of distinguishing patterns or characteristics. Following Pantel and Lin's approach, we can use the distribution of words across the unambiguous cases to predict the ambiguous cases in an unsupervised manner. 

Pantel and Lin define ambiguous cases of the form (V, N1, P, N2). 

They define unambiguous cases as of the forms (N1, P, N2) and (V, P, N2). 

I'm not sure I agree with this. What about... 

(P, N2, P, N3)? based on above, this is said to be unambiguous. I disagree. 
    sub (V, P, N2, P, N3)? 
    sub (N1, P, N2, P, N3)? 

I carried my golf clubs with new grips on graphite shafts with cavity-backed blades over my shoulder. 

I carried my golf clubs with new grips over my shoulder. 
I carried my golf clubs with new grips from the pro shop. 

I carried my golf bag with the new irons from the pro shop. 
I carried my golf bag across the 15th fairway to the pro shop. 
I carried my golf bag with my friend Jim across the 15th fairway. 
I carried my golf bag to the pro shop with my friend Jim. 

All of form...

N, V, N, P, N, P, N

Bah! Matches first rule. 

N, V, N, P, N, P, N, P, N

I'm calling that ambiguous. 

hence the only unambiguous cases would be of the form... 

(V, P, N2) or maybe
(N, P, N, V)

The man with the horse offered me a job. 

These heuristics must be ranked... first match fires, done. 
(cooler way to say that?)
disjunctive heuristics? 
decision trees? 

(V, N, P, N) => ambiguous (these cats)
(V, P, N) => unambiguous (yea. them cats)
They say...
N, P, N => unambiguous

I say...

(V, P, N, P, N) => ambiguous (I made this!)
I walked with my golf bag to the clubhouse. 
I walked with my golf bag in a pullcart. 

(P, N, P, N, P, N) => ambiguous

^(N, P, N, V) => unambiguous (I made this!)
The man with the beard sells tacos. 

(P, N, P, N) => unambiguous? (variation of N P N - them cats)
(P) => unknown (presumed unambiguous)
=> no P





In [1]:
import re
import os
import nltk

In [2]:
ex_parse = "(S (PP (IN In) (NP (QP (RB at) (RBS least) (CD one)) (NN image))) (NP (EX there)) (VP (VBZ is) (NP (NP (DT a) (JJ single) (NN tree)) (PP (IN with) (NP (JJ orange) (NNS flowers))) (PP (IN in) (NP (NP (NN front)) (PP (IN of) (NP (DT a) (NN church))))) (PP (IN with) (S (NP (DT the) (JJ open) (NN door)) (VP (VBG facing) (RB forward) (RB left)))))) (. .))"

In [3]:
print(ex_parse)

(S (PP (IN In) (NP (QP (RB at) (RBS least) (CD one)) (NN image))) (NP (EX there)) (VP (VBZ is) (NP (NP (DT a) (JJ single) (NN tree)) (PP (IN with) (NP (JJ orange) (NNS flowers))) (PP (IN in) (NP (NP (NN front)) (PP (IN of) (NP (DT a) (NN church))))) (PP (IN with) (S (NP (DT the) (JJ open) (NN door)) (VP (VBG facing) (RB forward) (RB left)))))) (. .))


In [4]:
pat1 = re.compile(r'[VNP]P')

In [5]:
pat1.findall('(S (VP (VBZ is)')

['VP']

In [12]:
ex_parse = "(S (PP (IN In) (NP (QP (RB at) (RBS least) (CD one)) (NN image))) (NP (EX there)) (VP (VBZ is) (NP (NP (DT a) (JJ single) (NN tree)) (PP (IN with) (NP (JJ orange) (NNS flowers))) (PP (IN in) (NP (NP (NN front)) (PP (IN of) (NP (DT a) (NN church))))) (PP (IN with) (S (NP (DT the) (JJ open) (NN door)) (VP (VBG facing) (RB forward) (RB left)))))) (. .))"
print(ex_parse)

(S (PP (IN In) (NP (QP (RB at) (RBS least) (CD one)) (NN image))) (NP (EX there)) (VP (VBZ is) (NP (NP (DT a) (JJ single) (NN tree)) (PP (IN with) (NP (JJ orange) (NNS flowers))) (PP (IN in) (NP (NP (NN front)) (PP (IN of) (NP (DT a) (NN church))))) (PP (IN with) (S (NP (DT the) (JJ open) (NN door)) (VP (VBG facing) (RB forward) (RB left)))))) (. .))


In [13]:
matches = re.findall(pat1,ex_parse)

In [15]:
phrase_pattern = ''.join([m[0] for m in matches])

In [16]:
phrase_pattern

'PNNVNNPNPNNPNPNV'

In [17]:
# NN -> N
new_phrase_pattern = 'N'.join(phrase_pattern.split('NN'))

In [18]:
new_phrase_pattern

'PNVNPNPNPNPNV'

In [21]:
# (V, N, P, N) => ambiguous (these cats) 
match_obj = re.search('VNPN', new_phrase_pattern)


In [22]:
ambiguous = False
if match_obj:
    ambiguous = True

In [23]:
if not ambiguous: 
    match_obj = re.search('VPNPN', new_phrase_pattern)
    if match_obj:
        ambiguous = True

In [25]:
if not ambiguous:
    match_obj = re.search('NPNPN', new_phrase_pattern)
    if match_obj:
        ambiguous = True

In [26]:
no_prep = False
if not ambiguous: 
    match_obj = re.search('P', new_phrase_pattern)
    if not match_obj:
        no_prep = True

In [20]:
# Result: ambiguous. STOP

In [27]:
x1="NPNPNP"
x2="NNZZFF"
m1=re.search('P',x1)
m2=re.search('P',x2)
if not m1:
    print("not m1")
if not m2:
    print("not m2")

not m2


In [36]:
from itertools import groupby

In [55]:
def has_pp_attachment_ambiguity(parse):
    # https://stackoverflow.com/questions/18799036/python-best-way-to-remove-duplicate-character-from-string
    step0 = re.sub(r'SBAR \(IN', 'PP (IN', parse)
    step1 = [m[0] for m in re.findall(r'[VNP]P',step0)]
    step2 = ''.join(step1)
    step3 = ''.join(ch for ch, _ in groupby(step2))
    phrase_pattern = step3
    
    # If no prep phrase, let's not waste time.
    if not re.search(r'P',phrase_pattern):
        if len(phrase_pattern)>0:
            print("No preps: {}".format(phrase_pattern))
            print(parse)
        return False
    
    # (V, N, P, N) => ambiguous (Pantel&Lin) 
    # Mary ate a salad with a fork. 
    # Mary ate a salad with croutons. 
    match_obj = re.search(r'VNPN', phrase_pattern)
    if match_obj:
        return True
    #(V, P, N, P, N) => ambiguous (I made this!) 
    #I walked with my golf bag to the clubhouse. 
    #I walked with my golf bag in a pullcart.
    match_obj = re.search(r'VPNPN', phrase_pattern)
    if match_obj:
        return True
    # (N, P, N, P, N) => ambiguous (I made this!)
    # In at least one image there is a single tree with orange flowers in front of a church with the open door facing forward left.
    # ... [tree with orange] flowers in front of a church ...
    # ... [tree with orange] flowers with five petals on thick branches... 
    # ([N, P, {N, P, N], P, N})
    match_obj = re.search(r'NPNPN', phrase_pattern)
    if match_obj:
        return True
    # (N, V, P, N) => ambiguous (discovered empirically)
    # ... women wearing white bikinis standing next to the water. 
    # (simplification)
    # ... A woman is wearing a bikini cooking on a gas stove. 
    # ... A woman is holding a spatula cooking in a white bikini. 
    if re.search(r'NVNVPN', phrase_pattern):
        return True
    
    # We have now passed all the checks for ambiguity. 
    
    #^(N, P, N, V) => unambiguous (I made this!)
    # The man with the beard sells tacos. 
    match_obj = re.search(r'^NPNV', phrase_pattern)
    if match_obj:
        print("Pattern match (^NPNV=un): {}".format(phrase_pattern))
    else:
        # ...in the image on the left...
        match_obj = re.search(r'PNPN', phrase_pattern)
        if match_obj:
            print("Pattern match (PNPN=un): {}".format(phrase_pattern))
        else:
            print("New pattern: {}".format(phrase_pattern))
    print(parse)
    return False

In [62]:
def get_pp_attachment_ambiguity(parse):
    # https://stackoverflow.com/questions/18799036/python-best-way-to-remove-duplicate-character-from-string
    step0 = re.sub(r'SBAR \(IN', 'PP (IN', parse)
    step1 = [m[0] for m in re.findall(r'[VNP]P',step0)]
    step2 = ''.join(step1)
    step3 = ''.join(ch for ch, _ in groupby(step2))
    phrase_pattern = step3
    
    # If no prep phrase, let's not waste time.
    if not re.search(r'P',phrase_pattern):
        if len(phrase_pattern)>0:
            print("No preps: {}".format(phrase_pattern))
            print(parse)
        return False, 'P'
    
    # (V, N, P, N) => ambiguous (Pantel&Lin) 
    # Mary ate a salad with a fork. 
    # Mary ate a salad with croutons. 
    match_obj = re.search(r'VNPN', phrase_pattern)
    if match_obj:
        return True, 'VNPN'
    #(V, P, N, P, N) => ambiguous (I made this!) 
    #I walked with my golf bag to the clubhouse. 
    #I walked with my golf bag in a pullcart.
    match_obj = re.search(r'VPNPN', phrase_pattern)
    if match_obj:
        return True, 'VPNPN'
    # (N, P, N, P, N) => ambiguous (I made this!)
    # In at least one image there is a single tree with orange flowers in front of a church with the open door facing forward left.
    # ... [tree with orange] flowers in front of a church ...
    # ... [tree with orange] flowers with five petals on thick branches... 
    # ([N, P, {N, P, N], P, N})
    match_obj = re.search(r'NPNPN', phrase_pattern)
    if match_obj:
        return True, 'NPNPN'
    # (N, V, P, N) => ambiguous (discovered empirically)
    # ... women wearing white bikinis standing next to the water. 
    # (simplification)
    # ... A woman is wearing a bikini cooking on a gas stove. 
    # ... A woman is holding a spatula cooking in a white bikini. 
    if re.search(r'NVNVPN', phrase_pattern):
        return True, 'NVNVPN'
    
    # We have now passed all the checks for ambiguity. 
    
    #^(N, P, N, V) => unambiguous (I made this!)
    # The man with the beard sells tacos. 
    match_obj = re.search(r'^NPNV', phrase_pattern)
    if match_obj:
        print("Pattern match (^NPNV=un): {}".format(phrase_pattern))
        print(parse)
        return False, '^NPNV'
    else:
        # ...in the image on the left...
        match_obj = re.search(r'PNPN', phrase_pattern)
        if match_obj:
            print("Pattern match (PNPN=un): {}".format(phrase_pattern))
            print(parse)
            return False, 'PNPN'
        else:
            print("New pattern: {}".format(phrase_pattern))
    print(parse)
    return False, phrase_pattern

In [56]:
parses = []
num_total = 0
num_found = 0
with open('../data/pp_ambiguity_parses.txt') as ppap_in:
    for line in ppap_in:
        num_total += 1
        if has_pp_attachment_ambiguity(line.strip()):
            num_found += 1

New pattern: NVPNVPNVN
(S (NP (DT A) (JJ large) (JJ brown) (NN dog)) (VP (VBZ is) (PP (IN on) (NP (DT the) (NN couch))) (ADVP (RB next) (PP (IN to) (NP (NP (DT a) (JJ blonde) (JJ haired) (NN woman)) (VP (VBG wearing) (NP (NNS pajamas))))))) (. .))
No preps: NVN
(S (NP (PRP it)) (VP (VBZ 's) (NP (JJ blue) (CC and) (NNP red) (NNP box))) (. .))


In [57]:
print(num_found, num_total)

92 94


In [ ]:

(P, N, P, N, P, N) => ambiguous

^(N, P, N, V) => unambiguous (I made this!) 

(P, N, P, N) => unambiguous? (variation of N P N - them cats) (P) => unknown (presumed unambiguous) => no P

In [58]:
parses = []
num_total = 0
num_found = 0
with open('../data/all_parses.txt') as ppap_in:
    for line in ppap_in:
        num_total += 1
        if has_pp_attachment_ambiguity(line.strip()):
            num_found += 1

New pattern: NPN
(NP (NP (QP (RB Only) (CD one)) (NN gerbil)) (PP (IN per) (NP (NN picture))) (. .) (_SP
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (QP (RB exactly) (CD three)) (JJ juvenile) (NNS cheetahs))) (. .) (_SP
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (QP (DT no) (JJR more) (IN than) (CD six)) (NNS birds))) (. .) (_SP
No preps: NVN
(S (NP (CD One) (NN dog)) (VP (VBZ is) (VP (VBG wearing) (NP (DT a) (NN leash)))) (. .) (_SP
Pattern match (PNPN=un): PNPNVPN
(S (PP (IN In) (NP (NP (DT the) (NN image)) (PP (IN to) (NP (DT the) (NN right))))) (, ,) (NP (DT the) (NN truck)) (VP (VBZ is) (VP (VBN equipped) (PP (IN with) (NP (DT a) (JJ red) (NN plow))))) (. .) (_SP
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (QP (DT no) (JJR more) (IN than) (CD three)) (NNS elephants))) (. .) (_SP
Pattern match (^NPNV=un): NPNVN
(S (NP (NP (PDT All) (DT the) (NNS birds)) (PP (IN in) (NP (DT these) (NNS images)))) (VP (VBP are) (NP (NNS adults) (_SP
No preps: NVNV
(S (S (NP (CD 

In [59]:
num_found

557

In [65]:
parses = []
num_total = 0
num_found = 0
ppaa_pats = {}
with open('../data/all_parses.txt') as ppap_in:
    for line in ppap_in:
        if len(line.strip())<8:
            continue
        num_total += 1
        has_ppaa, patstr = get_pp_attachment_ambiguity(line.strip())
        if has_ppaa:
            num_found += 1
        if not patstr in ppaa_pats:
            ppaa_pats[patstr] = 0
        ppaa_pats[patstr] += 1

New pattern: NPN
(NP (NP (QP (RB Only) (CD one)) (NN gerbil)) (PP (IN per) (NP (NN picture))) (. .) (_SP
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (QP (RB exactly) (CD three)) (JJ juvenile) (NNS cheetahs))) (. .) (_SP
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (QP (DT no) (JJR more) (IN than) (CD six)) (NNS birds))) (. .) (_SP
No preps: NVN
(S (NP (CD One) (NN dog)) (VP (VBZ is) (VP (VBG wearing) (NP (DT a) (NN leash)))) (. .) (_SP
Pattern match (PNPN=un): PNPNVPN
(S (PP (IN In) (NP (NP (DT the) (NN image)) (PP (IN to) (NP (DT the) (NN right))))) (, ,) (NP (DT the) (NN truck)) (VP (VBZ is) (VP (VBN equipped) (PP (IN with) (NP (DT a) (JJ red) (NN plow))))) (. .) (_SP
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (QP (DT no) (JJR more) (IN than) (CD three)) (NNS elephants))) (. .) (_SP
Pattern match (^NPNV=un): NPNVN
(S (NP (NP (PDT All) (DT the) (NNS birds)) (PP (IN in) (NP (DT these) (NNS images)))) (VP (VBP are) (NP (NNS adults) (_SP
No preps: NVNV
(S (S (NP (CD 

In [66]:
num_total

825

In [67]:
num_found

557

In [68]:
ppaa_pats

{'VNPN': 364,
 'NPN': 3,
 'P': 150,
 'PNPN': 6,
 '^NPNV': 51,
 'VPNPN': 105,
 'NPNPN': 31,
 'PNVNVNV': 1,
 'NVNVPN': 57,
 'NVPNVPN': 6,
 'NVPN': 30,
 'PNVNVN': 2,
 'PNVN': 5,
 'PNVNV': 3,
 'VPN': 1,
 'PNVPN': 4,
 'NVPNVPNVN': 1,
 'NVPNVN': 2,
 'NVPNV': 1,
 'PNVPNVPN': 1,
 'PN': 1}

In [109]:
all_parses = []
with open('../data/annotated_dev.parse') as ppap_in:
    for line in ppap_in:
        all_parses.append(line.strip())

In [110]:
ppaa_parses = []
with open('../data/pp_ambiguity.parse') as ppap_in:
    for line in ppap_in:
        ppaa_parses.append(line.strip())

In [111]:
pos_ambiguous_parses = [parse for parse in all_parses if parse in ppaa_parses]

In [112]:
len(pos_ambiguous_parses)

94

In [113]:
annotated_parses = [(parse, parse in ppaa_parses) for parse in all_parses]

In [116]:
my_predictions = [has_pp_attachment_ambiguity(parse) for parse in all_parses]

New pattern: NPN
(NP (NP (QP (RB Only) (CD one)) (NN gerbil)) (PP (IN per) (NP (NN picture))) (. .))
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (QP (RB exactly) (CD three)) (JJ juvenile) (NNS cheetahs))) (. .))
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (QP (DT no) (JJR more) (IN than) (CD six)) (NNS birds))) (. .))
No preps: NVN
(S (NP (CD One) (NN dog)) (VP (VBZ is) (VP (VBG wearing) (NP (DT a) (NN leash)))) (. .))
Pattern match (PNPN=un): PNPNVPN
(S (PP (IN In) (NP (NP (DT the) (NN image)) (PP (IN to) (NP (DT the) (NN right))))) (, ,) (NP (DT the) (NN truck)) (VP (VBZ is) (VP (VBN equipped) (PP (IN with) (NP (DT a) (JJ red) (NN plow))))) (. .))
No preps: NVN
(S (NP (EX There)) (VP (VBP are) (NP (QP (DT no) (JJR more) (IN than) (CD three)) (NNS elephants))) (. .))
Pattern match (^NPNV=un): NPNVN
(S (NP (NP (PDT All) (DT the) (NNS birds)) (PP (IN in) (NP (DT these) (NNS images)))) (VP (VBP are) (NP (NNS adults))))
No preps: NVNV
(S (S (NP (CD One) (NN dog) (NN sled) (N

In [117]:
their_predictions = [p[1] for p in annotated_parses]

In [119]:
import numpy as np
import sklearn


In [124]:
mine = np.array([int(boo) for boo in my_predictions])
theirs = np.array([int(boo) for boo in their_predictions])
false_positives = [all_parses[i] for i in range(len(all_parses)) if mine[i]>theirs[i]]

In [125]:
len(false_positives)

463

In [128]:
fp_stuff = [get_pp_attachment_ambiguity(parse) for parse in false_positives]

In [129]:
len(fp_stuff)

463

In [131]:
from collections import Counter

In [132]:
Counter([fp[1] for fp in fp_stuff])

Counter({'VNPN': 307, 'VPNPN': 73, 'NPNPN': 31, 'NVNVPN': 52})

In [133]:
interesting = [false_positives[i] for i in range(len(false_positives)) if fp_stuff[i][1]=='VNPN']

In [134]:
len(interesting)

307

In [139]:
interesting[141]

'(S (NP (DT A) (NN person)) (VP (VBZ is) (VP (VBG holding) (NP (DT the) (NN crab)) (PP (IN in) (NP (NP (DT the) (NN image)) (PP (IN on) (NP (DT the) (NN left))))))) (. .))'

In [107]:
def search_parses(regex, parses):
    for parse in parses:
        if re.search(regex, parse):
            yield parse

In [108]:
for parse in search_parses(r'bobby', all_parses):
    print(parse)

(S (NP (DT a) (NNP bobby) (NN pin)) (VP (VBZ is) (VP (VBN pinned) (PP (IN on) (NP (NP (DT a) (NN sweater)) (PP (IN with) (NP (NP (DT a) (JJ collared) (NN shirt)) (PP (IN underneath) (NP)


In [99]:
ppaa_parses[9]

'(S (S (NP (DT Each) (NN image)) (VP (VBZ contains) (NP (CD one) (JJ non) (JJ -) (JJ standing) (NN spaniel)))) (, ,) (CC and) (S (NP (CD one) (NN image)) (VP (VBZ features) (NP (NP (DT a) (NN spaniel)) (PP (IN with) (NP (NP (NN fur)) (PP (IN in) (NP (NP (NNS shades)) (PP (IN of) (NP (NN brown))))))) (PP (PP (IN with) (NP (DT a) (JJ white) (NN chest) (NN stripe))) (, ,) (VP (VBD posed) (S (VP (VBG sitting) (ADJP (JJ upright))))))))) (. .))'

In [101]:
for parse in all_parses:
    if re.search(r'(spaniel.*){2}',parse):
        print(parse)

(S (S (NP (DT Each) (NN image)) (VP (VBZ contains) (NP (CD one) (JJ non) (JJ -) (JJ standing) (NN spaniel)))) (, ,) (CC and) (S (NP (CD one) (NN image)) (VP (VBZ features) (NP (NP (DT a) (NN spaniel)) (PP (IN with) (NP (NP (NN fur)) (PP (IN in) (NP (NP (NNS shades)) (PP (IN of) (NP (NN brown))))))) (PP (IN with) (NP (DT a) (JJ white) (NN chest) (NN stripe))) (, ,) (VP (VBD posed) (S (VP (VBG sitting) (ADJP (JJ upright)))))))) (. .)


In [102]:
new_parses = ['{})'.format(parse) for parse in all_parses]
all_parses = new_parses